## Simple line fitting using mean squared error cost function

In [22]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ----------------------------
# Data Generation
# ----------------------------
def generate_data(m=2, c=3, n=50, noise=5, seed=42):
    np.random.seed(seed)
    X = np.linspace(0, 10, n)
    y = m * X + c + np.random.randn(n) * noise
    return X, y

# ----------------------------
# Gradient Descent
# ----------------------------
def gradient_descent(X, y, lr=0.001, iters=100, init_m=0, init_c=0):
    m, c = init_m, init_c
    n = len(y)
    losses, params = [], []

    for _ in range(iters):
        y_pred = m * X + c
        error = y_pred - y

        dm = (2/n) * np.dot(error, X)
        dc = (2/n) * np.sum(error)

        m -= lr * dm
        c -= lr * dc

        loss = np.mean(error**2)
        losses.append(loss)
        params.append((m, c, y_pred))
    return params, losses

# ----------------------------
# Parameters
# ----------------------------
true_m, true_c = 2, 5
noise = 4
n_points = 50
lr = 0.001
iterations = 100
init_m, init_c = 0, 0

X, y = generate_data(m=true_m, c=true_c, n=n_points, noise=noise)
params, losses = gradient_descent(X, y, lr=lr, iters=iterations,
                                  init_m=init_m, init_c=init_c)

# ----------------------------
# Helper: Build projection trace
# ----------------------------
def make_projection_trace(X, y, y_pred):
    x_proj, y_proj = [], []
    for xi, yi, ypi in zip(X, y, y_pred):
        x_proj += [xi, xi, None]
        y_proj += [yi, ypi, None]
    return x_proj, y_proj

# ----------------------------
# Initial traces (static + placeholders)
# ----------------------------
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("Data and Best Fit Line", "Loss vs Iterations"))

# Trace 0 = static data points (will never change)
fig.add_trace(go.Scatter(x=X, y=y, mode='markers',
                         marker=dict(color='blue'), name='Data'),
              row=1, col=1)

# Trace 1 = best-fit line (animated)
fig.add_trace(go.Scatter(x=X, y=params[0][2], mode='lines',
                         line=dict(color='red'), name='Best Fit Line'),
              row=1, col=1)

# Trace 2 = projections (animated)
x_proj, y_proj = make_projection_trace(X, y, params[0][2])
fig.add_trace(go.Scatter(x=x_proj, y=y_proj, mode='lines',
                         line=dict(color='gray', dash='dash'),
                         showlegend=False),
              row=1, col=1)

# Trace 3 = loss curve (animated)
fig.add_trace(go.Scatter(x=[0], y=[losses[0]], mode='lines',
                         line=dict(color='green'), name='Loss'),
              row=1, col=2)

# ----------------------------
# Frames (update traces 1, 2, 3 only)
# ----------------------------
frames = []
for i, (m, c, y_pred) in enumerate(params):
    # Projections
    x_proj, y_proj = make_projection_trace(X, y, y_pred)

    frames.append(go.Frame(
        data=[
            go.Scatter(x=X, y=y_pred),              # updates trace 1
            go.Scatter(x=x_proj, y=y_proj),         # updates trace 2
            go.Scatter(x=list(range(i+1)), y=losses[:i+1])  # updates trace 3
        ],
        traces=[1, 2, 3],  # which traces to update
        name=str(i)
    ))

# ----------------------------
# Layout + Controls
# ----------------------------
fig.update_layout(
    title="Linear Regression with Gradient Descent",
    updatemenus=[{
        "type": "buttons",
        "buttons": [
            {"label": "Play", "method": "animate",
             "args": [None, {"frame": {"duration": 100, "redraw": True},
                             "fromcurrent": True}]},
            {"label": "Pause", "method": "animate",
             "args": [[None], {"frame": {"duration": 0, "redraw": False},
                               "mode": "immediate"}]}
        ],
    }],
    sliders=[{
        "steps": [{"args": [[str(i)], {"frame": {"duration": 0, "redraw": True},
                                        "mode": "immediate"}],
                   "label": str(i), "method": "animate"} for i in range(iterations)]
    }]
)

fig.frames = frames
fig.show()


In [23]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ----------------------------
# Data Generation
# ----------------------------
def generate_data(a=2, b=1, c=5, n=100, noise=5, seed=42):
    np.random.seed(seed)
    X = np.random.uniform(-10, 10, n)
    Y = np.random.uniform(-10, 10, n)
    Z = a * X + b * Y + c + np.random.randn(n) * noise
    return X, Y, Z

# ----------------------------
# Gradient Descent for Plane z = a*x + b*y + c
# ----------------------------
def gradient_descent(X, Y, Z, lr=0.0001, iters=100, init_a=0, init_b=0, init_c=0):
    a, b, c = init_a, init_b, init_c
    n = len(Z)
    losses, params = [], []

    for _ in range(iters):
        Z_pred = a * X + b * Y + c
        error = Z_pred - Z

        # Gradients
        da = (2/n) * np.dot(error, X)
        db = (2/n) * np.dot(error, Y)
        dc = (2/n) * np.sum(error)

        a -= lr * da
        b -= lr * db
        c -= lr * dc

        loss = np.mean(error**2)
        losses.append(loss)
        params.append((a, b, c))
    return params, losses

# ----------------------------
# Parameters
# ----------------------------
true_a, true_b, true_c = 2, -1, 5
noise = 5
n_points = 100
lr = 0.0005
iterations = 100
init_a, init_b, init_c = 0, 0, 0

# Generate data
X, Y, Z = generate_data(a=true_a, b=true_b, c=true_c,
                        n=n_points, noise=noise)
params, losses = gradient_descent(X, Y, Z, lr=lr, iters=iterations,
                                  init_a=init_a, init_b=init_b, init_c=init_c)

# Mesh grid for plane plotting
xx, yy = np.meshgrid(np.linspace(min(X), max(X), 20),
                     np.linspace(min(Y), max(Y), 20))

# ----------------------------
# Initial figure setup
# ----------------------------
fig = make_subplots(rows=1, cols=2,
                    specs=[[{"type": "scene"}, {"type": "xy"}]],
                    subplot_titles=("3D Data and Plane Fit", "Loss vs Iterations"))

# Static scatter (trace 0)
fig.add_trace(go.Scatter3d(x=X, y=Y, z=Z, mode='markers',
                           marker=dict(size=4, color='blue'), name="Data"),
              row=1, col=1)

# Dummy plane (trace 1)
zz0 = params[0][0] * xx + params[0][1] * yy + params[0][2]
fig.add_trace(go.Surface(x=xx, y=yy, z=zz0, colorscale="Reds",
                         opacity=0.6, showscale=False, name="Plane"),
              row=1, col=1)

# Dummy loss curve (trace 2)
fig.add_trace(go.Scatter(x=[0], y=[losses[0]], mode='lines',
                         line=dict(color='green'), name="Loss"),
              row=1, col=2)

# ----------------------------
# Animation frames
# ----------------------------
frames = []
for i, (a, b, c) in enumerate(params):
    zz = a * xx + b * yy + c
    frames.append(go.Frame(
        data=[
            go.Surface(x=xx, y=yy, z=zz, colorscale="Reds",
                       opacity=0.6, showscale=False),  # updates plane (trace 1)
            go.Scatter(x=list(range(i+1)), y=losses[:i+1])  # updates loss (trace 2)
        ],
        traces=[1, 2],
        name=str(i)
    ))

# ----------------------------
# Layout + Controls
# ----------------------------
fig.update_layout(
    title="3D Linear Regression (Plane) with Gradient Descent",
    scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
    updatemenus=[{
        "type": "buttons",
        "buttons": [
            {"label": "Play", "method": "animate",
             "args": [None, {"frame": {"duration": 100, "redraw": True},
                             "fromcurrent": True}]},
            {"label": "Pause", "method": "animate",
             "args": [[None], {"frame": {"duration": 0, "redraw": False},
                               "mode": "immediate"}]}
        ],
    }],
    sliders=[{
        "steps": [{"args": [[str(i)], {"frame": {"duration": 0, "redraw": True},
                                        "mode": "immediate"}],
                   "label": str(i), "method": "animate"} for i in range(iterations)]
    }]
)

fig.frames = frames
fig.show()


## Effect of regularization 

In [26]:
import numpy as np
from sklearn.linear_model import Lasso, Ridge, LinearRegression
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ----------------------------
# Synthetic Data Generation
# ----------------------------
def generate_dense_data(n_samples=100, n_features=30, noise=0.1, seed=42):
    np.random.seed(seed)
    X = np.random.randn(n_samples, n_features)
    
    # Dense true weights (all non-zero)
    w_true = np.random.uniform(-3, 3, n_features)
    
    y = X @ w_true + np.random.randn(n_samples) * noise
    return X, y, w_true

# ----------------------------
# Fit models for a given lambda
# ----------------------------
def fit_models(X, y, lam):
    # L1 (Lasso)
    lasso = Lasso(alpha=lam, fit_intercept=False, max_iter=5000)
    lasso.fit(X, y)
    w_l1 = lasso.coef_
    
    # No regularization (OLS)
    ols = LinearRegression(fit_intercept=False)
    ols.fit(X, y)
    w_ols = ols.coef_
    
    # L2 (Ridge)
    ridge = Ridge(alpha=lam, fit_intercept=False)
    ridge.fit(X, y)
    w_l2 = ridge.coef_
    
    return w_l1, w_ols, w_l2

# ----------------------------
# Data
# ----------------------------
X, y, w_true = generate_dense_data(n_samples=100, n_features=30, noise=1.0)

# Feature labels
feature_names = [f"x{i+1}" for i in range(X.shape[1])]

# ----------------------------
# Prepare slider frames
# ----------------------------
lambda_values = np.logspace(-3, 2, 20)  # 0.001 → 100
frames = []

for lam in lambda_values:
    w_l1, w_ols, w_l2 = fit_models(X, y, lam)
    
    frames.append(go.Frame(
        data=[
            go.Bar(x=feature_names, y=w_l1, marker_color="red"),
            go.Bar(x=feature_names, y=w_ols, marker_color="blue"),
            go.Bar(x=feature_names, y=w_l2, marker_color="green"),
        ],
        name=f"{lam:.4f}"
    ))

# ----------------------------
# Initial model fit
# ----------------------------
w_l1, w_ols, w_l2 = fit_models(X, y, lambda_values[0])

# ----------------------------
# Figure layout
# ----------------------------
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=("L1 Regularization (Lasso)",
                                    "No Regularization (OLS)",
                                    "L2 Regularization (Ridge)"))

# Initial coefficient bars
fig.add_trace(go.Bar(x=feature_names, y=w_l1, marker_color="red"), row=1, col=1)
fig.add_trace(go.Bar(x=feature_names, y=w_ols, marker_color="blue"), row=2, col=1)
fig.add_trace(go.Bar(x=feature_names, y=w_l2, marker_color="green"), row=3, col=1)

# ----------------------------
# Slider setup
# ----------------------------
steps = []
for lam in lambda_values:
    steps.append(dict(
        method="animate",
        args=[[f"{lam:.4f}"], {"frame": {"duration": 500, "redraw": True},
                               "mode": "immediate"}],
        label=f"{lam:.4f}"
    ))

sliders = [dict(
    active=0,
    steps=steps,
    currentvalue={"prefix": "Lambda: "}
)]

fig.update_layout(
    title="Effect of L1 vs L2 Regularization on Dense Weights",
    sliders=sliders,
    height=900,
    showlegend=False
)

fig.frames = frames
fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.pipeline import make_pipeline
import ipywidgets as widgets
from ipywidgets import interact

# --- Generate sample data ---
np.random.seed(42)
x = np.linspace(0, 10, 20)
y = (x - 5) ** 3 + 3 * (x - 6) ** 2
noise = np.random.normal(0, 16, y.shape)
y = y + noise
X = x.reshape(-1, 1)

deg = 21

# --- OLS (λ=0, no regularization) ---
ols = make_pipeline(
    PolynomialFeatures(degree=deg),
    StandardScaler(with_mean=False),
    LinearRegression()
)
ols.fit(X, y)
y_pred_ols = ols.predict(X)

# --- Function to update plot based on λ ---
def plot_with_lambda(lam=0.0):
    if lam == 0:
        y_pred_ridge = y_pred_ols
        y_pred_lasso = y_pred_ols
    else:
        ridge = make_pipeline(
            PolynomialFeatures(degree=deg),
            StandardScaler(with_mean=False),
            Ridge(alpha=lam)
        )
        ridge.fit(X, y)
        y_pred_ridge = ridge.predict(X)

        lasso = make_pipeline(
            PolynomialFeatures(degree=deg),
            StandardScaler(with_mean=False),
            Lasso(alpha=lam, max_iter=200000)
        )
        lasso.fit(X, y)
        y_pred_lasso = lasso.predict(X)

    # Build interactive plot
    fig = go.Figure()

    # Data points
    fig.add_trace(go.Scatter(x=x, y=y, mode='markers',
                             name='Data', marker=dict(color='black')))

    # OLS baseline
    fig.add_trace(go.Scatter(x=x, y=y_pred_ols, mode='lines',
                             name='OLS (λ=0)', line=dict(color='blue')))

    # Ridge curve
    fig.add_trace(go.Scatter(x=x, y=y_pred_ridge, mode='lines',
                             name=f'Ridge (λ={lam})', line=dict(color='green')))

    # Lasso curve
    fig.add_trace(go.Scatter(x=x, y=y_pred_lasso, mode='lines',
                             name=f'Lasso (λ={lam})', line=dict(color='red')))

    fig.update_layout(
        template="plotly_white",
        title=f"Polynomial Regression with Ridge & Lasso (λ={lam})",
        xaxis_title="x",
        yaxis_title="y"
    )
    fig.show()

# --- Interactive slider ---
interact(plot_with_lambda, lam=widgets.FloatLogSlider(
    value=0.0,
    base=10,
    min=-2,  # 10^-2 = 0.01
    max=3,   # 10^3 = 1000
    step=0.01,
    description="λ (lambda)"
))


interactive(children=(FloatLogSlider(value=0.01, description='λ (lambda)', max=3.0, min=-2.0, step=0.01), Outp…

<function __main__.plot_with_lambda(lam=0.0)>

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso, LinearRegression
import ipywidgets as widgets
from ipywidgets import interact

# --- Generate sample data ---
np.random.seed(42)
x = np.linspace(0, 10, 20)
y = (x - 5) ** 3 + 3 * (x - 6) ** 2
noise = np.random.normal(0, 100, y.shape)
y = y + noise
X = x.reshape(-1, 1)

deg = 9
poly = PolynomialFeatures(degree=deg, include_bias=False)
X_poly = poly.fit_transform(X)

# --- Feature scaling ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_poly)

def denormalize(coef_scaled, intercept_scaled, scaler):
    """Convert coefficients/intercept from scaled to original space."""
    coef_orig = coef_scaled / scaler.scale_
    intercept_orig = intercept_scaled - np.sum(scaler.mean_ / scaler.scale_ * coef_scaled)
    return coef_orig, intercept_orig

# --- OLS baseline ---
ols = LinearRegression()
ols.fit(X_scaled, y)
y_pred_ols = ols.predict(X_scaled)
ols_coef, ols_intercept = denormalize(ols.coef_, ols.intercept_, scaler)

degrees = np.arange(len(ols_coef))

# --- Function to update plots ---
def plot_with_lambda(lam=0.0):
    if lam == 0:
        y_pred_ridge = y_pred_ols
        y_pred_lasso = y_pred_ols
        ridge_coef, ridge_intercept = ols_coef, ols_intercept
        lasso_coef, lasso_intercept = ols_coef, ols_intercept
    else:
        # Ridge
        ridge = Ridge(alpha=lam)
        ridge.fit(X_scaled, y)
        y_pred_ridge = ridge.predict(X_scaled)
        ridge_coef, ridge_intercept = denormalize(ridge.coef_, ridge.intercept_, scaler)

        # Lasso
        lasso = Lasso(alpha=lam, max_iter=200000)
        lasso.fit(X_scaled, y)
        y_pred_lasso = lasso.predict(X_scaled)
        lasso_coef, lasso_intercept = denormalize(lasso.coef_, lasso.intercept_, scaler)

    # --- Subplots ---
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=("Polynomial Fits", "Regression Coefficients"),
        row_heights=[0.6, 0.4],
        shared_xaxes=False
    )

    # Top: Data + Fits
    fig.add_trace(go.Scatter(x=x, y=y, mode='markers',
                             name='Data', marker=dict(color='black')),
                  row=1, col=1)

    fig.add_trace(go.Scatter(x=x, y=y_pred_ols, mode='lines',
                             name='OLS (λ=0)', line=dict(color='blue')),
                  row=1, col=1)

    fig.add_trace(go.Scatter(x=x, y=y_pred_ridge, mode='lines',
                             name=f'Ridge (λ={lam})', line=dict(color='green')),
                  row=1, col=1)

    fig.add_trace(go.Scatter(x=x, y=y_pred_lasso, mode='lines',
                             name=f'Lasso (λ={lam})', line=dict(color='red')),
                  row=1, col=1)

    # Bottom: Grouped bar chart of coefficients
    fig.add_trace(go.Bar(x=degrees, y=ols_coef, name="OLS (λ=0)",
                         marker_color="blue"), row=2, col=1)

    fig.add_trace(go.Bar(x=degrees, y=ridge_coef, name=f"Ridge (λ={lam})",
                         marker_color="green"), row=2, col=1)

    fig.add_trace(go.Bar(x=degrees, y=lasso_coef, name=f"Lasso (λ={lam})",
                         marker_color="red"), row=2, col=1)

    fig.update_layout(
        template="plotly_white",
        title=f"Polynomial Regression with Ridge & Lasso (λ={lam})",
        height=800,
        barmode="group"
    )

    fig.update_xaxes(title="Polynomial degree", row=2, col=1)
    fig.update_yaxes(title="Coefficient value", row=2, col=1)

    fig.show()

# --- Interactive slider ---
interact(plot_with_lambda, lam=widgets.FloatLogSlider(
    value=0.0,
    base=10,
    min=-2,  # 10^-2 = 0.01
    max=3,   # 10^3 = 1000
    step=0.01,
    description="λ (lambda)"
))


interactive(children=(FloatLogSlider(value=0.01, description='λ (lambda)', max=3.0, min=-2.0, step=0.01), Outp…

<function __main__.plot_with_lambda(lam=0.0)>